In [3]:
import torch
import time
# Задание 3: Сравнение производительности CPU vs CUDA

# 3.1 Подготовка данных
def create_tensors():
    return {
        "64x1024x1024": torch.randn(64, 1024, 1024),
        "128x512x512": torch.randn(128, 512, 512),
        "256x256x256": torch.randn(256, 256, 256),
    }

# 3.2 Функция измерения времени на CPU
def time_op_cpu(op, tensor, repeats=3):
    times = []
    for _ in range(repeats):
        start = time.time()
        op(tensor)
        times.append(time.time() - start)
    return (sum(times) / repeats) * 1000  # время в миллисекундах

# 3.2 Функция измерения времени на GPU
def time_op_cuda(op, tensor, repeats=10):
    torch.cuda.synchronize()
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    start_event.record()
    for _ in range(repeats):
        op(tensor)
    end_event.record()

    torch.cuda.synchronize()
    return start_event.elapsed_time(end_event) / repeats  # время в миллисекундах

# 3.3 Сравнение операций
def benchmark(tensor, device):
    tensor = tensor.to(device)  # переносим тензор

    # операции для измерения времени
    ops = {
        "Матричное умножение": lambda x: torch.matmul(x, x.transpose(-1, -2)),
        "Сложение": lambda x: x + x,
        "Умножение": lambda x: x * x,
        "Транспонирование": lambda x: x.transpose(-1, -2),
        "Сумма": lambda x: x.sum(),
    }

    results = {}
    for name, op in ops.items():
        if device.type == 'cuda':
            ms = time_op_cuda(op, tensor)
        else:
            ms = time_op_cpu(op, tensor)
        results[name] = ms
    return results

# Красивый вывод в таблицу
def print_results(cpu_results, gpu_results):
    print(f"{'Операция':<20} | {'CPU (мс)':>9} | {'GPU (мс)':>9} | {'Ускорение':>9}")
    print("-" * 60)
    for op in cpu_results:
        cpu_time = cpu_results[op]
        gpu_time = gpu_results.get(op, None)
        if gpu_time and gpu_time > 0:
            speedup = cpu_time / gpu_time
            print(f"{op:<20} | {cpu_time:9.2f} | {gpu_time:9.2f} | {speedup:9.2f}x")
        else:
            print(f"{op:<20} | {cpu_time:9.2f} | {'-':>9} | {'-':>9}")

def main():
    tensors = create_tensors()
    device_cpu = torch.device('cpu')
    device_cuda = torch.device('cuda') if torch.cuda.is_available() else None

    print(f"CUDA доступна: {torch.cuda.is_available()}")
    if device_cuda:
        print(f"GPU: {torch.cuda.get_device_name(0)}")

    # 3.3 Сравнение операций
    for size_name, tensor in tensors.items():
        print(f"\nРазмер: {size_name}")
        cpu_results = benchmark(tensor, device_cpu)
        gpu_results = benchmark(tensor, device_cuda) if device_cuda else {}
        print_results(cpu_results, gpu_results)

if __name__ == "__main__":
    main()


CUDA доступна: True
GPU: Tesla T4

Размер: 64x1024x1024
Операция             |  CPU (мс) |  GPU (мс) | Ускорение
------------------------------------------------------------
Матричное умножение  |   2086.27 |     42.45 |     49.15x
Сложение             |    153.50 |      2.21 |     69.55x
Умножение            |    154.51 |      2.21 |     69.90x
Транспонирование     |      0.02 |      0.01 |      3.08x
Сумма                |     23.00 |      0.99 |     23.34x

Размер: 128x512x512
Операция             |  CPU (мс) |  GPU (мс) | Ускорение
------------------------------------------------------------
Матричное умножение  |    516.29 |     12.02 |     42.96x
Сложение             |     74.64 |      1.11 |     67.46x
Умножение            |     75.91 |      1.11 |     68.62x
Транспонирование     |      0.02 |      0.01 |      2.52x
Сумма                |     11.04 |      0.52 |     21.13x

Размер: 256x256x256
Операция             |  CPU (мс) |  GPU (мс) | Ускорение
-----------------------------

# ***Анализ результатов***

**Какие операции получают наибольшее ускорение на GPU?**

Наибольшее ускорение демонстрируют матричное умножение и поэлементные операции — они выполняются в десятки раз быстрее по сравнению с CPU.

**Почему некоторые операции могут быть медленнее на GPU?**

Транспонирование почти не ускоряется, потому что оно очень простое и быстрое само по себе. А время, которое тратится на запуск работы на GPU и ожидание результата, сводит на нет всю выгоду.

**Как размер матриц влияет на ускорение?**

Чем больше размер матриц, тем выше ускорение, так как GPU эффективнее использует свои ресурсы при работе с большими объемами данных.

**Что происходит при передаче данных между CPU и GPU?**

Передача данных между CPU и GPU занимает значительное время, что может снизить общую производительность, поэтому стоит минимизировать такие операции.